In [1]:
!git clone --recursive https://github.com/Microsoft/LightGBM
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

Cloning into 'LightGBM'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 20629 (delta 47), reused 37 (delta 15), pack-reused 20531
Receiving objects: 100% (20629/20629), 16.40 MiB | 35.06 MiB/s, done.
Resolving deltas: 100% (15062/15062), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728/21728)

In [60]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame, concat
!pip install import_ipynb
import import_ipynb
from google.colab import drive
drive.mount('/content/drive')
from datetime import date
import lightgbm as lgbm
from lightgbm import LGBMRegressor
import time
import statsmodels.api as sm
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split
from tqdm import tnrange, tqdm_notebook
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import os
from sklearn.decomposition import PCA

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import math
from math import radians

In [5]:
#### 파라미터 ####

# 하루의 틱
ticks = 48

# 예측에 사용할 일수
days = 3                # 이 부분을 바꿀 수 있다.
n_days = ticks*days

# 미래 예측할 일수
future_days = 2
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']

# 사용할 변수
what_to_left = ['Time', 'WS', 'DHI','DNI','RH','T','TARGET', 'Day']

n_features = len(what_to_left)
n_obs = n_days * n_features # 7일 예측 기준으로 7(일 수) * 48(틱 수) * 6(변수 개수)

In [6]:
# 데이터 불러오기 

raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/train/train.csv')
submission = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/sample_submission.csv')
submission.shape

(7776, 10)

In [7]:
def drop_clms(dataset):
  dataset['Time'] = dataset['Hour'] + dataset['Minute']*(0.5/30)
  dataset['Date'] = dataset['Day']%365
  dataset.drop('Day', axis=1, inplace=True)
  
  temp = list()
  for i in range(0, len(dataset), 48):
    temp += [24-(list(dataset.DHI[i:i+48]).count(0)*0.5)]*48 

  dataset["SH"] = temp

  return dataset

def cos_time(dataset):
  dataset['sin_time'] = np.sin(2*np.pi*dataset.Time/24)
  dataset['cos_time'] = np.cos(2*np.pi*dataset.Time/24)
  return dataset

In [8]:
def get_yday(when):
  HP = date(2020,1,1)
  results = (when-HP).days
  return results

In [9]:
def 절기24(Date):
  target = 0
  for i in list24:
    if Date < i:
      target = list24.index(i) - 1
      break
  if Date < 5:
    target = 23
  return target

In [30]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def train_to_supervised(train, target, n_in):

    clmns = list(train.columns)

    # 기타 칼럼은 전과 같이 들어갈 것.
    scaled_lst = clmns

    scaled_df = train[scaled_lst]
    target_df = target

    # 미래 몇 번째 항목을 가져올 것인가
    future = [48, 96]

    ### 만약에 스케일링을 하고 싶다면 ###
    # scaled_df 데이터 프레임만 스케일링 하고, 절기랑 TARGET 데이터는 그냥 두면 된다.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    n_vars2 = 1 if type(target_df) is list else target_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # 48과 96 후의 타겟 데이터 2개 붙이기.
    # forecast sequence (t, t+1, ... t+n)
    for i in future:
        cols.append(target_df.shift(-i))
        if i == 0:
            names += [('TARGET%d(t)' % (j+1)) for j in range(n_vars2)]
        else:
            names += [('TARGET%d(t+%d)' % (j+1, i)) for j in range(n_vars2)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [29]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def test_to_supervised(train, n_in):

    clmns = list(train.columns)

    # 기타 칼럼은 전과 같이 들어갈 것.
    scaled_lst = clmns
    scaled_df = train[scaled_lst]

    ### 만약에 스케일링을 하고 싶다면 ###
    # testset의 스케일링은 구조가 상당히 까다로우므로....... 일단 나중에 하기로 함.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]
    
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [12]:
소한 = get_yday(date(2020,1,6))
대한 = get_yday(date(2020,1,20))
입춘 = get_yday(date(2020,2,4))
우수 = get_yday(date(2020,2,19))
경칩 = get_yday(date(2020,3,6))
춘분 = get_yday(date(2020,3,21))
청명 = get_yday(date(2020,4,5))
곡우 = get_yday(date(2020,4,20))
입하 = get_yday(date(2020,5,6))
소만 = get_yday(date(2020,5,21))
망종 = get_yday(date(2020,6,6))
하지 = get_yday(date(2020,6,22))
소서 = get_yday(date(2020,7,7))
대서 = get_yday(date(2020,7,23))
입추 = get_yday(date(2020,8,8))
처서 = get_yday(date(2020,8,23))
백로 = get_yday(date(2020,9,8))
추분 = get_yday(date(2020,9,23))
한로 = get_yday(date(2020,10,8))
상강 = get_yday(date(2020,10,24))
입동 = get_yday(date(2020,11,8))
소설 = get_yday(date(2020,11,22))
대설 = get_yday(date(2020,12,7))
동지 = get_yday(date(2020,12,22))

list24 = [소한, 대한, 입춘, 우수, 경칩, 춘분, 청명, 곡우, 입하, 소만, 망종, 하지, 소서, 대서, 입추, 처서, 백로, 추분, 한로, 상강, 입동, 소설, 대설, 동지]

In [13]:
m = 60

일출 = [7+47/m, 7+44/m, 7+34/m, 7+18/m, 6+56/m, 6+35/m, 6+12/m, 5+51/m, 5+33/m, 5+19/m, 5+11/m, 5+11/m, 5+17/m, 5+27/m, 5+41/m, 5+54/m, 6+8/m, 6+20/m, 6+33/m, 6+47/m, 7+3/m, 7+20/m, 7+33/m, 7+45/m]
일몰 = [17+28/m, 17+42/m, 17+58/m, 18+15/m, 18+30/m, 18+44/m, 18+58/m, 19+12/m, 19+25/m, 19+39/m, 19+50/m, 19+57/m, 19+56/m, 19+50/m, 19+35/m, 19+15/m, 18+51/m, 18+28/m, 18+6/m, 17+45/m, 17+28/m, 17+17/m, 17+13/m, 17+17/m]
경사각 = [32.92, 36.83, 40.75, 44.67, 48.58, 52.5, 56.42, 60.33, 64.25, 68.16, 72.01, 76, 72.1, 68.16, 64.25, 60.33, 56.42, 52.5, 48.58, 44.67, 40.75, 36.83, 32.92, 29]
남중 = [12+36/m, 12+41/m, 12+44/m, 12+44/m, 12+42/m, 12+38/m, 12+34/m, 12+29/m, 12+27/m, 12+27/m, 12+29/m, 12+32/m, 12+35/m, 12+37/m, 12+36/m, 12+33/m, 12+29/m, 12+23/m, 12+18/m, 12+15/m, 12+14/m, 12+16/m, 12+22/m, 12+29/m]
print(len(일출), len(일몰), len(경사각), len(남중))

24 24 24 24


In [14]:
def HRA(DHI, DNI, season, hour):
  # 위도(latitude) 기준을 일단 임의로 대전으로 설정 (위도 36.19~36.2도)
  latitude = radians(36.2)
  season = int(season)
  # 절기별 대한민국의 경사각
  tilt = radians(경사각[season])

  # 절기별 대한민국 대전의 태양 남중시각
  hra = radians(15*(hour - 남중[season]))

  # 구하려는 알파
  elevation = np.arcsin(np.sin(tilt) * np.sin(latitude) + np.cos(tilt) * np.cos(latitude) * np.cos(hra))

  # 천정각(Zenith Angle)은 90 - 알파
  zenith = radians(90) - elevation

  # GHI는 DHI + DNI * cos(천정각)
  ghi = DHI + DNI *np.cos(zenith)

  return ghi

In [15]:
def not_minus(dataset):
  for i in range(0, len(dataset.index)):
    for j in range(0, len(dataset.columns)):
      K = dataset.iloc[i,j]
      if K < 0:
        dataset.iloc[i,j] = 0
  return dataset

In [16]:
dataset = raw_data.copy()
dataset2 = drop_clms(dataset)
dataset2.drop(['Hour', 'Minute'], axis =1, inplace=True)
dataset2

,DHI,DNI,WS,RH,T,TARGET,Time,Date,SH
0,0,0,1.5,69.08,-12,0.0,0.0,0,9.0
1,0,0,1.5,69.06,-12,0.0,0.5,0,9.0
2,0,0,1.6,71.78,-12,0.0,1.0,0,9.0
3,0,0,1.6,71.75,-12,0.0,1.5,0,9.0
4,0,0,1.6,75.20,-12,0.0,2.0,0,9.0
...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,21.5,364,9.0
52556,0,0,2.4,66.79,-4,0.0,22.0,364,9.0
52557,0,0,2.2,66.78,-4,0.0,22.5,364,9.0
52558,0,0,2.1,67.72,-4,0.0,23.0,364,9.0


In [17]:
dataset2['season'] = dataset2.apply(lambda x: 절기24(x['Date']), axis = 1)
dataset3 = cos_time(dataset2)
dataset4 = dataset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]
# dataset5 = dataset4.iloc[n_days:,:-1]
# goals = pd.DataFrame(dataset4.Goal, index = dataset4.index)
dataset4

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time
0,0,0,1.5,69.08,-12,0.0,9.0,0.000000,1.000000,23,0.0
1,0,0,1.5,69.06,-12,0.0,9.0,0.130526,0.991445,23,0.5
2,0,0,1.6,71.78,-12,0.0,9.0,0.258819,0.965926,23,1.0
3,0,0,1.6,71.75,-12,0.0,9.0,0.382683,0.923880,23,1.5
4,0,0,1.6,75.20,-12,0.0,9.0,0.500000,0.866025,23,2.0
...,...,...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,9.0,-0.608761,0.793353,0,21.5
52556,0,0,2.4,66.79,-4,0.0,9.0,-0.500000,0.866025,0,22.0
52557,0,0,2.2,66.78,-4,0.0,9.0,-0.382683,0.923880,0,22.5
52558,0,0,2.1,67.72,-4,0.0,9.0,-0.258819,0.965926,0,23.0


In [18]:
dataset5= dataset4.copy()

dataset5['GHI'] = dataset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

In [19]:
dataset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,1.6,88.11,-11,0.0,9.0,0.258819,0.965926,23,1.0,0.0


In [20]:
dataset6 = dataset5[['DHI', 'DNI', 'WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']]
dataset6.head(10)

,DHI,DNI,WS,RH,T,SH,sin_time,cos_time,GHI,TARGET
0,0,0,1.5,69.08,-12,9.0,0.000000,1.000000,0.0,0.0
1,0,0,1.5,69.06,-12,9.0,0.130526,0.991445,0.0,0.0
2,0,0,1.6,71.78,-12,9.0,0.258819,0.965926,0.0,0.0
3,0,0,1.6,71.75,-12,9.0,0.382683,0.923880,0.0,0.0
4,0,0,1.6,75.20,-12,9.0,0.500000,0.866025,0.0,0.0
5,0,0,1.5,69.29,-11,9.0,0.608761,0.793353,0.0,0.0
6,0,0,1.5,72.56,-11,9.0,0.707107,0.707107,0.0,0.0
7,0,0,1.4,72.55,-11,9.0,0.793353,0.608761,0.0,0.0
8,0,0,1.3,74.62,-11,9.0,0.866025,0.500000,0.0,0.0
9,0,0,1.3,74.61,-11,9.0,0.923880,0.382683,0.0,0.0


In [51]:
temp_X = dataset6.iloc[:,:-1]
temp_Y = pd.DataFrame(dataset6.iloc[:,-1])

scaler = MinMaxScaler(feature_range=(-1, 1))

scaler.fit(dataset6)

dataset7 = pd.DataFrame(scaler.transform(dataset6))

dataset7.columns = dataset6.columns
dataset7

,DHI,DNI,WS,RH,T,SH,sin_time,cos_time,GHI,TARGET
0,-1.0,-1.0,-0.750000,0.330808,-0.740741,-0.833333,0.000000,1.000000,-1.0,-1.0
1,-1.0,-1.0,-0.750000,0.330376,-0.740741,-0.833333,0.130526,0.991445,-1.0,-1.0
2,-1.0,-1.0,-0.733333,0.389244,-0.740741,-0.833333,0.258819,0.965926,-1.0,-1.0
3,-1.0,-1.0,-0.733333,0.388594,-0.740741,-0.833333,0.382683,0.923880,-1.0,-1.0
4,-1.0,-1.0,-0.733333,0.463262,-0.740741,-0.833333,0.500000,0.866025,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...
52555,-1.0,-1.0,-0.600000,0.365869,-0.444444,-0.833333,-0.608761,0.793353,-1.0,-1.0
52556,-1.0,-1.0,-0.600000,0.281247,-0.444444,-0.833333,-0.500000,0.866025,-1.0,-1.0
52557,-1.0,-1.0,-0.633333,0.281030,-0.444444,-0.833333,-0.382683,0.923880,-1.0,-1.0
52558,-1.0,-1.0,-0.650000,0.301374,-0.444444,-0.833333,-0.258819,0.965926,-1.0,-1.0


## dataset6 가지고 동일하게 to_supervised 함수를 써서 나누면 됩니다.
### 나머지 변수 합치기, 인코딩은 유진이가 해줄 것...

In [46]:
dataset8 = train_to_supervised(dataset7, temp_Y,n_days)
dataset8

,DHI(t-144),DNI(t-144),WS(t-144),RH(t-144),T(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHI(t-144),TARGET(t-144),DHI(t-143),DNI(t-143),WS(t-143),RH(t-143),T(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHI(t-143),TARGET(t-143),DHI(t-142),DNI(t-142),WS(t-142),RH(t-142),T(t-142),SH(t-142),sin_time(t-142),cos_time(t-142),GHI(t-142),TARGET(t-142),DHI(t-141),DNI(t-141),WS(t-141),RH(t-141),T(t-141),SH(t-141),sin_time(t-141),cos_time(t-141),GHI(t-141),TARGET(t-141),...,WS(t-4),RH(t-4),T(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),DHI(t-3),DNI(t-3),WS(t-3),RH(t-3),T(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),DHI(t-2),DNI(t-2),WS(t-2),RH(t-2),T(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),DHI(t-1),DNI(t-1),WS(t-1),RH(t-1),T(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1),TARGET1(t+48),TARGET1(t+96)
144,-1.0,-1.0,-0.750000,0.330808,-0.740741,-0.833333,0.000000,1.000000,-1.0,-1.0,-1.0,-1.0,-0.750000,0.330376,-0.740741,-0.833333,0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.733333,0.389244,-0.740741,-0.833333,0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.733333,0.388594,-0.740741,-0.833333,0.382683,0.923880,-1.0,-1.0,...,-0.933333,0.547452,-0.777778,-0.833333,-0.500000,0.866025,-1.0,-1.0,-1.0,-1.0,-0.883333,0.547235,-0.777778,-0.833333,-0.382683,0.923880,-1.0,-1.0,-1.0,-1.0,-0.816667,0.590737,-0.777778,-0.833333,-0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.766667,0.590521,-0.777778,-0.833333,-0.130526,0.991445,-1.0,-1.0,0.0,0.0
145,-1.0,-1.0,-0.750000,0.330376,-0.740741,-0.833333,0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.733333,0.389244,-0.740741,-0.833333,0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.733333,0.388594,-0.740741,-0.833333,0.382683,0.923880,-1.0,-1.0,-1.0,-1.0,-0.733333,0.463262,-0.740741,-0.833333,0.500000,0.866025,-1.0,-1.0,...,-0.883333,0.547235,-0.777778,-0.833333,-0.382683,0.923880,-1.0,-1.0,-1.0,-1.0,-0.816667,0.590737,-0.777778,-0.833333,-0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.766667,0.590521,-0.777778,-0.833333,-0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.700000,0.518667,-0.814815,-0.833333,0.000000,1.000000,-1.0,-1.0,0.0,0.0
146,-1.0,-1.0,-0.733333,0.389244,-0.740741,-0.833333,0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.733333,0.388594,-0.740741,-0.833333,0.382683,0.923880,-1.0,-1.0,-1.0,-1.0,-0.733333,0.463262,-0.740741,-0.833333,0.500000,0.866025,-1.0,-1.0,-1.0,-1.0,-0.750000,0.335353,-0.703704,-0.833333,0.608761,0.793353,-1.0,-1.0,...,-0.816667,0.590737,-0.777778,-0.833333,-0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.766667,0.590521,-0.777778,-0.833333,-0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.700000,0.518667,-0.814815,-0.833333,0.000000,1.000000,-1.0,-1.0,-1.0,-1.0,-0.666667,0.518883,-0.777778,-0.833333,0.130526,0.991445,-1.0,-1.0,0.0,0.0
147,-1.0,-1.0,-0.733333,0.388594,-0.740741,-0.833333,0.382683,0.923880,-1.0,-1.0,-1.0,-1.0,-0.733333,0.463262,-0.740741,-0.833333,0.500000,0.866025,-1.0,-1.0,-1.0,-1.0,-0.750000,0.335353,-0.703704,-0.833333,0.608761,0.793353,-1.0,-1.0,-1.0,-1.0,-0.750000,0.406125,-0.703704,-0.833333,0.707107,0.707107,-1.0,-1.0,...,-0.766667,0.590521,-0.777778,-0.833333,-0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.700000,0.518667,-0.814815,-0.833333,0.000000,1.000000,-1.0,-1.0,-1.0,-1.0,-0.666667,0.518883,-0.777778,-0.833333,0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.633333,0.517152,-0.777778,-0.833333,0.258819,0.965926,-1.0,-1.0,0.0,0.0
148,-1.0,-1.0,-0.733333,0.463262,-0.740741,-0.833333,0.500000,0.866025,-1.0,-1.0,-1.0,-1.0,-0.750000,0.335353,-0.703704,-0.833333,0.608761,0.793353,-1.0,-1.0,-1.0,-1.0,-0.750000,0.406125,-0.703704,-0.833333,0.707107,0.707107,-1.0,-1.0,-1.0,-1.0,-0.766667,0.405908,-0.703704,-0.833333,0.793353,0.608761,-1.0,-1.0,...,-0.700000,0.518667,-0.814815,-0.833333,0.000000,1.000000,-1.0,-1.0,-1.0,-1.0,-0.666667,0.518883,-0.777778,-0.833333,0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.633333,0.517152,-0.777778,-0.833333,0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.633333,0.384699,-0.777778,-0.833333,0.382683,0.923880,-1.0,-1.0,0.0,0.0
.

In [72]:
X = dataset8.iloc[:,:-2]

y_1 = pd.DataFrame(dataset8.iloc[:,-2])
y_2 = pd.DataFrame(dataset8.iloc[:,-1])
X

,DHI(t-144),DNI(t-144),WS(t-144),RH(t-144),T(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHI(t-144),TARGET(t-144),DHI(t-143),DNI(t-143),WS(t-143),RH(t-143),T(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHI(t-143),TARGET(t-143),DHI(t-142),DNI(t-142),WS(t-142),RH(t-142),T(t-142),SH(t-142),sin_time(t-142),cos_time(t-142),GHI(t-142),TARGET(t-142),DHI(t-141),DNI(t-141),WS(t-141),RH(t-141),T(t-141),SH(t-141),sin_time(t-141),cos_time(t-141),GHI(t-141),TARGET(t-141),...,DHI(t-4),DNI(t-4),WS(t-4),RH(t-4),T(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),DHI(t-3),DNI(t-3),WS(t-3),RH(t-3),T(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),DHI(t-2),DNI(t-2),WS(t-2),RH(t-2),T(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),DHI(t-1),DNI(t-1),WS(t-1),RH(t-1),T(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1)
144,-1.0,-1.0,-0.750000,0.330808,-0.740741,-0.833333,0.000000,1.000000,-1.0,-1.0,-1.0,-1.0,-0.750000,0.330376,-0.740741,-0.833333,0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.733333,0.389244,-0.740741,-0.833333,0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.733333,0.388594,-0.740741,-0.833333,0.382683,0.923880,-1.0,-1.0,...,-1.0,-1.0,-0.933333,0.547452,-0.777778,-0.833333,-0.500000,0.866025,-1.0,-1.0,-1.0,-1.0,-0.883333,0.547235,-0.777778,-0.833333,-0.382683,0.923880,-1.0,-1.0,-1.0,-1.0,-0.816667,0.590737,-0.777778,-0.833333,-0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.766667,0.590521,-0.777778,-0.833333,-0.130526,0.991445,-1.0,-1.0
145,-1.0,-1.0,-0.750000,0.330376,-0.740741,-0.833333,0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.733333,0.389244,-0.740741,-0.833333,0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.733333,0.388594,-0.740741,-0.833333,0.382683,0.923880,-1.0,-1.0,-1.0,-1.0,-0.733333,0.463262,-0.740741,-0.833333,0.500000,0.866025,-1.0,-1.0,...,-1.0,-1.0,-0.883333,0.547235,-0.777778,-0.833333,-0.382683,0.923880,-1.0,-1.0,-1.0,-1.0,-0.816667,0.590737,-0.777778,-0.833333,-0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.766667,0.590521,-0.777778,-0.833333,-0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.700000,0.518667,-0.814815,-0.833333,0.000000,1.000000,-1.0,-1.0
146,-1.0,-1.0,-0.733333,0.389244,-0.740741,-0.833333,0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.733333,0.388594,-0.740741,-0.833333,0.382683,0.923880,-1.0,-1.0,-1.0,-1.0,-0.733333,0.463262,-0.740741,-0.833333,0.500000,0.866025,-1.0,-1.0,-1.0,-1.0,-0.750000,0.335353,-0.703704,-0.833333,0.608761,0.793353,-1.0,-1.0,...,-1.0,-1.0,-0.816667,0.590737,-0.777778,-0.833333,-0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.766667,0.590521,-0.777778,-0.833333,-0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.700000,0.518667,-0.814815,-0.833333,0.000000,1.000000,-1.0,-1.0,-1.0,-1.0,-0.666667,0.518883,-0.777778,-0.833333,0.130526,0.991445,-1.0,-1.0
147,-1.0,-1.0,-0.733333,0.388594,-0.740741,-0.833333,0.382683,0.923880,-1.0,-1.0,-1.0,-1.0,-0.733333,0.463262,-0.740741,-0.833333,0.500000,0.866025,-1.0,-1.0,-1.0,-1.0,-0.750000,0.335353,-0.703704,-0.833333,0.608761,0.793353,-1.0,-1.0,-1.0,-1.0,-0.750000,0.406125,-0.703704,-0.833333,0.707107,0.707107,-1.0,-1.0,...,-1.0,-1.0,-0.766667,0.590521,-0.777778,-0.833333,-0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.700000,0.518667,-0.814815,-0.833333,0.000000,1.000000,-1.0,-1.0,-1.0,-1.0,-0.666667,0.518883,-0.777778,-0.833333,0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.633333,0.517152,-0.777778,-0.833333,0.258819,0.965926,-1.0,-1.0
148,-1.0,-1.0,-0.733333,0.463262,-0.740741,-0.833333,0.500000,0.866025,-1.0,-1.0,-1.0,-1.0,-0.750000,0.335353,-0.703704,-0.833333,0.608761,0.793353,-1.0,-1.0,-1.0,-1.0,-0.750000,0.406125,-0.703704,-0.833333,0.707107,0.707107,-1.0,-1.0,-1.0,-1.0,-0.766667,0.405908,-0.703704,-0.833333,0.793353,0.608761,-1.0,-1.0,...,-1.0,-1.0,-0.700000,0.518667,-0.814815,-0.833333,0.000000,1.000000,-1.0,-1.0,-1.0,-1.0,-0.666667,0.518883,-0.777778,-0.833333,0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.633333,0.517152,-0.777778,-0.833333,0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.633333,0.384699,-0.777778,-0.833333,0.382683,0.923880,-1.0,-1.0
.

In [73]:
Pika = 0.99
pca = PCA(n_components= Pika)
pca.fit(X)
X = pd.DataFrame(pca.transform(X), index = X.index)

In [74]:
X

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240
144,0.755775,12.796784,14.034195,4.399141,-0.360725,2.935693,0.084624,1.065436,-2.291515,-0.209710,-2.440160,1.620591,-3.153234,-1.095710,0.572037,-0.415125,-2.270484,-3.696489,-0.999419,-0.957457,-1.077649,0.007571,-0.213875,-0.638643,-1.452264,1.211756,-1.550171,-1.474658,0.230043,0.827991,-0.623780,-0.512188,0.738770,0.074426,-0.054625,-0.082570,-1.339999,0.566558,1.065868,-0.952160,...,0.098516,-0.103013,-0.149652,-0.110776,-0.243847,-0.231916,0.127521,0.024002,0.388500,0.220204,-0.226288,-0.041287,0.074045,-0.188111,-0.296725,-0.177432,0.188909,0.142094,-0.204889,-0.148431,-0.443872,0.262879,0.034425,0.022183,-0.026752,0.159179,0.233694,0.318063,-0.121166,-0.057289,-0.181287,-0.129914,0.258457,0.319565,0.215073,-0.087548,-0.393104,0.172726,-0.161990,0.194461
145,-0.923200,12.790784,14.038948,4.424898,0.419457,2.900462,0.151470,1.071613,-2.261029,-0.488345,-2.410090,1.525590,-3.320304,-0.790939,0.631037,-0.124669,-2.068166,-3.832632,-0.771901,-1.032346,-1.217264,-0.083455,0.131025,-0.824722,-0.876332,1.421629,-1.572900,-1.505331,0.081615,1.031992,-0.269910,-0.598419,0.844244,0.120503,-0.049750,-0.120694,-1.029247,0.078551,1.343846,-1.021985,...,0.025366,0.122629,0.143508,-0.093065,0.207182,0.114265,0.114600,0.046710,-0.268285,-0.433418,0.240931,-0.028089,-0.061821,0.117241,-0.179352,-0.315117,-0.184353,-0.223092,0.144877,0.265059,0.235731,-0.438376,0.039175,-0.005806,0.056441,-0.134282,0.140438,-0.431027,-0.008465,0.021115,0.173000,0.341904,-0.133098,-0.204182,-0.042796,0.121041,0.421968,-0.193957,0.067573,-0.176837
146,-2.586876,12.564899,14.042418,4.446989,1.164697,2.664902,0.213542,1.078541,-2.193032,-0.763235,-2.381742,1.420199,-3.471332,-0.481958,0.674884,0.166856,-1.831747,-3.883922,-0.560194,-1.068929,-1.327317,-0.205341,0.466845,-0.980179,-0.246728,1.527018,-1.549340,-1.529156,-0.077475,0.970193,0.117126,-0.677458,0.875054,0.162021,-0.053470,-0.141736,-0.657830,-0.457928,1.443791,-1.073090,...,-0.181436,-0.163186,-0.177205,0.074686,-0.154088,0.041324,0.061348,-0.300909,0.116130,0.422273,-0.281553,-0.045339,0.002001,-0.027252,-0.054640,-0.401967,0.052142,0.245540,-0.013246,-0.321583,0.031281,0.487971,-0.030614,-0.050563,-0.006688,0.112433,0.298483,0.406423,0.003740,-0.084474,0.009245,-0.342594,-0.119772,0.174511,0.143538,-0.017922,-0.423119,0.189617,0.031092,0.333493
147,-4.206518,12.121968,14.045494,4.463292,1.824644,2.245891,0.266865,1.086780,-2.088621,-1.031313,-2.351420,1.307287,-3.603004,-0.171769,0.707406,0.455226,-1.565072,-3.854896,-0.371025,-1.067756,-1.393976,-0.354870,0.775675,-1.095970,0.402114,1.519938,-1.481814,-1.539472,-0.177569,0.658384,0.430577,-0.741520,0.830286,0.184318,-0.065714,-0.141212,-0.282356,-0.975644,1.355525,-1.098532,...,0.311711,0.198083,0.187898,0.050285,0.094495,-0.086862,0.190414,0.305575,0.143376,-0.364962,0.223363,0.047991,-0.113261,-0.151044,-0.242432,-0.289719,-0.089445,-0.198396,-0.213511,0.319018,-0.291416,-0.390780,0.042989,0.086638,0.014199,-0.164622,0.227696,-0.420438,0.070496,-0.045679,-0.023917,0.285573,0.267284,0.002868,-0.034554,0.031169,0.444522,-0.178276,-0.122905,-0.137949
148,-5.753657,11.467800,14.047824,4.469900,2.357607,1.673375,0.295980,1.091808,-1.948127,-1.289541,-2.315698,1.190736,-3.713726,0.143234,0.730490,0.739864,-1.269088,-3.752765,-0.211827,-1.036257,-1.407940,-0.514094,1.041704,-1.164430,1.034020,1.399735,-1.364077,-1.529732,-0.160913,0.178862,0.587265,-0.779258,0.716343,0.188964,-0.084586,-0.124095,0.052273,-1.403971,1.091495,-1.087356,...,-0.349795,-0.121028,-0.049142,0.217415,-0.065094,0.021612,0.032719,-0.384021,-0.289661,0.134805,-0.214105,0.031970,-0.003956,0.200821,-0.243020,-0.325847,-0.022304,0.197787,0.369259,-0.173720,0.441537,0.189713,-0.052724,-0.162545,0.022223,0.2187

# testset은 이미 24절기(season)가 들어있으므로, 약간 다르게 전처리를 해 줘야 함.
## (season) 구하는 과정이 생략됨.
## 이하는 testset을 구하는 코드

In [33]:
file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(1) + '.csv'
new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(1) + '.csv'
file_name = str(1) + '.csv'

# adj_test에서 가져와야 하므로 new_path에서 불러온다
temp = pd.read_csv(new_path)
temp

,Unnamed: 0,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,Time,24
0,0,0,0,0,0,0,2.8,70.34,-6.1,0.0,0.0,22
1,1,0,0,30,0,0,2.9,71.97,-6.4,0.0,0.5,22
2,2,0,1,0,0,0,2.9,69.77,-6.7,0.0,1.0,22
3,3,0,1,30,0,0,2.9,71.95,-7.1,0.0,1.5,22
4,4,0,2,0,0,0,2.9,69.48,-7.4,0.0,2.0,22
...,...,...,...,...,...,...,...,...,...,...,...,...
331,331,6,21,30,0,0,1.7,43.65,-5.2,0.0,21.5,22
332,332,6,22,0,0,0,1.5,40.62,-4.9,0.0,22.0,22
333,333,6,22,30,0,0,1.5,40.62,-4.9,0.0,22.5,22
334,334,6,23,0,0,0,1.5,39.21,-4.9,0.0,23.0,22


In [34]:
temp.drop('Unnamed: 0', axis = 1, inplace = True)

In [35]:
testset = temp.copy()

# 24로 저장한 24절기 season으로 이름 바꾸기
testset.rename(columns = {'24' : 'season'}, inplace = True)


# 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
testset2 = drop_clms(testset)
testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
testset3 = cos_time(testset2)
testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]
testset4

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time
0,0,0,2.8,70.34,-6.1,0.0,9.5,0.000000,1.000000,22,0.0
1,0,0,2.9,71.97,-6.4,0.0,9.5,0.130526,0.991445,22,0.5
2,0,0,2.9,69.77,-6.7,0.0,9.5,0.258819,0.965926,22,1.0
3,0,0,2.9,71.95,-7.1,0.0,9.5,0.382683,0.923880,22,1.5
4,0,0,2.9,69.48,-7.4,0.0,9.5,0.500000,0.866025,22,2.0
...,...,...,...,...,...,...,...,...,...,...,...
331,0,0,1.7,43.65,-5.2,0.0,9.5,-0.608761,0.793353,22,21.5
332,0,0,1.5,40.62,-4.9,0.0,9.5,-0.500000,0.866025,22,22.0
333,0,0,1.5,40.62,-4.9,0.0,9.5,-0.382683,0.923880,22,22.5
334,0,0,1.5,39.21,-4.9,0.0,9.5,-0.258819,0.965926,22,23.0


In [36]:
testset5 = testset4.copy()
testset5['GHI'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

In [37]:
testset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,2.1,61.33,-10.8,0.0,9.5,0.258819,0.965926,22,1.0,0.0


In [38]:
#### testset과 dataset이 같은 모양인지 중간점검
dataset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,1.6,88.11,-11,0.0,9.0,0.258819,0.965926,23,1.0,0.0


In [43]:
testset6 = testset5[['DHI', 'DNI', 'WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']]
testset6.shape

(336, 10)

In [52]:
testset7 = pd.DataFrame(scaler.transform(testset6))

In [55]:
testset7.columns = testset6.columns

In [56]:
testset8 = test_to_supervised(testset7, n_days)
testset8

,DHI(t-144),DNI(t-144),WS(t-144),RH(t-144),T(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHI(t-144),TARGET(t-144),DHI(t-143),DNI(t-143),WS(t-143),RH(t-143),T(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHI(t-143),TARGET(t-143),DHI(t-142),DNI(t-142),WS(t-142),RH(t-142),T(t-142),SH(t-142),sin_time(t-142),cos_time(t-142),GHI(t-142),TARGET(t-142),DHI(t-141),DNI(t-141),WS(t-141),RH(t-141),T(t-141),SH(t-141),sin_time(t-141),cos_time(t-141),GHI(t-141),TARGET(t-141),...,DHI(t-4),DNI(t-4),WS(t-4),RH(t-4),T(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),DHI(t-3),DNI(t-3),WS(t-3),RH(t-3),T(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),DHI(t-2),DNI(t-2),WS(t-2),RH(t-2),T(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),DHI(t-1),DNI(t-1),WS(t-1),RH(t-1),T(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1)
144,-1.0,-1.0,-0.533333,0.358078,-0.522222,-0.666667,0.000000,1.000000,-1.0,-1.0,-1.0,-1.0,-0.516667,0.393356,-0.533333,-0.666667,0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.516667,0.345742,-0.544444,-0.666667,0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.516667,0.392923,-0.559259,-0.666667,0.382683,0.923880,-1.0,-1.0,...,-1.0,-1.0,-0.333333,-0.037334,-0.462963,-0.666667,-0.500000,0.866025,-1.0,-1.0,-1.0,-1.0,-0.350000,-0.038632,-0.462963,-0.666667,-0.382683,0.923880,-1.0,-1.0,-1.0,-1.0,-0.350000,-0.001623,-0.459259,-0.666667,-0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.350000,-0.010497,-0.455556,-0.666667,-0.130526,0.991445,-1.0,-1.0
145,-1.0,-1.0,-0.516667,0.393356,-0.533333,-0.666667,0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.516667,0.345742,-0.544444,-0.666667,0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.516667,0.392923,-0.559259,-0.666667,0.382683,0.923880,-1.0,-1.0,-1.0,-1.0,-0.516667,0.339465,-0.570370,-0.666667,0.500000,0.866025,-1.0,-1.0,...,-1.0,-1.0,-0.350000,-0.038632,-0.462963,-0.666667,-0.382683,0.923880,-1.0,-1.0,-1.0,-1.0,-0.350000,-0.001623,-0.459259,-0.666667,-0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.350000,-0.010497,-0.455556,-0.666667,-0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.350000,0.024564,-0.451852,-0.666667,0.000000,1.000000,-1.0,-1.0
146,-1.0,-1.0,-0.516667,0.345742,-0.544444,-0.666667,0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.516667,0.392923,-0.559259,-0.666667,0.382683,0.923880,-1.0,-1.0,-1.0,-1.0,-0.516667,0.339465,-0.570370,-0.666667,0.500000,0.866025,-1.0,-1.0,-1.0,-1.0,-0.516667,0.396819,-0.588889,-0.666667,0.608761,0.793353,-1.0,-1.0,...,-1.0,-1.0,-0.350000,-0.001623,-0.459259,-0.666667,-0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.350000,-0.010497,-0.455556,-0.666667,-0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.350000,0.024564,-0.451852,-0.666667,0.000000,1.000000,-1.0,-1.0,-1.0,-1.0,-0.366667,0.015691,-0.448148,-0.666667,0.130526,0.991445,-1.0,-1.0
147,-1.0,-1.0,-0.516667,0.392923,-0.559259,-0.666667,0.382683,0.923880,-1.0,-1.0,-1.0,-1.0,-0.516667,0.339465,-0.570370,-0.666667,0.500000,0.866025,-1.0,-1.0,-1.0,-1.0,-0.516667,0.396819,-0.588889,-0.666667,0.608761,0.793353,-1.0,-1.0,-1.0,-1.0,-0.516667,0.345525,-0.603704,-0.666667,0.707107,0.707107,-1.0,-1.0,...,-1.0,-1.0,-0.350000,-0.010497,-0.455556,-0.666667,-0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.350000,0.024564,-0.451852,-0.666667,0.000000,1.000000,-1.0,-1.0,-1.0,-1.0,-0.366667,0.015691,-0.448148,-0.666667,0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.383333,0.065253,-0.444444,-0.666667,0.258819,0.965926,-1.0,-1.0
148,-1.0,-1.0,-0.516667,0.339465,-0.570370,-0.666667,0.500000,0.866025,-1.0,-1.0,-1.0,-1.0,-0.516667,0.396819,-0.588889,-0.666667,0.608761,0.793353,-1.0,-1.0,-1.0,-1.0,-0.516667,0.345525,-0.603704,-0.666667,0.707107,0.707107,-1.0,-1.0,-1.0,-1.0,-0.516667,0.393356,-0.618519,-0.666667,0.793353,0.608761,-1.0,-1.0,...,-1.0,-1.0,-0.350000,0.024564,-0.451852,-0.666667,0.000000,1.000000,-1.0,-1.0,-1.0,-1.0,-0.366667,0.015691,-0.448148,-0.666667,0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.383333,0.065253,-0.444444,-0.666667,0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.400000,0.045558,-0.437037,-0.666667,0.382683,0.923880,-

In [57]:
testset9 = testset8.iloc[-96:,:]
testset9

,DHI(t-144),DNI(t-144),WS(t-144),RH(t-144),T(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHI(t-144),TARGET(t-144),DHI(t-143),DNI(t-143),WS(t-143),RH(t-143),T(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHI(t-143),TARGET(t-143),DHI(t-142),DNI(t-142),WS(t-142),RH(t-142),T(t-142),SH(t-142),sin_time(t-142),cos_time(t-142),GHI(t-142),TARGET(t-142),DHI(t-141),DNI(t-141),WS(t-141),RH(t-141),T(t-141),SH(t-141),sin_time(t-141),cos_time(t-141),GHI(t-141),TARGET(t-141),...,DHI(t-4),DNI(t-4),WS(t-4),RH(t-4),T(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),DHI(t-3),DNI(t-3),WS(t-3),RH(t-3),T(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),DHI(t-2),DNI(t-2),WS(t-2),RH(t-2),T(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),DHI(t-1),DNI(t-1),WS(t-1),RH(t-1),T(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1)
240,-1.0,-1.0,-0.883333,0.223677,-0.696296,-0.666667,0.000000,1.000000,-1.0,-1.0,-1.0,-1.0,-0.883333,0.268694,-0.711111,-0.666667,0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.883333,0.265447,-0.725926,-0.666667,0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.883333,0.300292,-0.737037,-0.666667,0.382683,0.923880,-1.0,-1.0,...,-1.0,-1.0,-0.883333,0.322801,-0.611111,-0.666667,-0.500000,0.866025,-1.0,-1.0,-1.0,-1.0,-0.900000,0.346175,-0.618519,-0.666667,-0.382683,0.923880,-1.0,-1.0,-1.0,-1.0,-0.916667,0.305486,-0.629630,-0.666667,-0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.933333,0.328644,-0.637037,-0.666667,-0.130526,0.991445,-1.0,-1.0
241,-1.0,-1.0,-0.883333,0.268694,-0.711111,-0.666667,0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.883333,0.265447,-0.725926,-0.666667,0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.883333,0.300292,-0.737037,-0.666667,0.382683,0.923880,-1.0,-1.0,-1.0,-1.0,-0.883333,0.273022,-0.744444,-0.666667,0.500000,0.866025,-1.0,-1.0,...,-1.0,-1.0,-0.900000,0.346175,-0.618519,-0.666667,-0.382683,0.923880,-1.0,-1.0,-1.0,-1.0,-0.916667,0.305486,-0.629630,-0.666667,-0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.933333,0.328644,-0.637037,-0.666667,-0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.700000,0.070663,-0.874074,-0.666667,0.000000,1.000000,-1.0,-1.0
242,-1.0,-1.0,-0.883333,0.265447,-0.725926,-0.666667,0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.883333,0.300292,-0.737037,-0.666667,0.382683,0.923880,-1.0,-1.0,-1.0,-1.0,-0.883333,0.273022,-0.744444,-0.666667,0.500000,0.866025,-1.0,-1.0,-1.0,-1.0,-0.866667,0.296396,-0.751852,-0.666667,0.608761,0.793353,-1.0,-1.0,...,-1.0,-1.0,-0.916667,0.305486,-0.629630,-0.666667,-0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.933333,0.328644,-0.637037,-0.666667,-0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.700000,0.070663,-0.874074,-0.666667,0.000000,1.000000,-1.0,-1.0,-1.0,-1.0,-0.750000,0.070663,-0.874074,-0.666667,0.130526,0.991445,-1.0,-1.0
243,-1.0,-1.0,-0.883333,0.300292,-0.737037,-0.666667,0.382683,0.923880,-1.0,-1.0,-1.0,-1.0,-0.883333,0.273022,-0.744444,-0.666667,0.500000,0.866025,-1.0,-1.0,-1.0,-1.0,-0.866667,0.296396,-0.751852,-0.666667,0.608761,0.793353,-1.0,-1.0,-1.0,-1.0,-0.866667,0.275187,-0.759259,-0.666667,0.707107,0.707107,-1.0,-1.0,...,-1.0,-1.0,-0.933333,0.328644,-0.637037,-0.666667,-0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.700000,0.070663,-0.874074,-0.666667,0.000000,1.000000,-1.0,-1.0,-1.0,-1.0,-0.750000,0.070663,-0.874074,-0.666667,0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.783333,0.060058,-0.870370,-0.666667,0.258819,0.965926,-1.0,-1.0
244,-1.0,-1.0,-0.883333,0.273022,-0.744444,-0.666667,0.500000,0.866025,-1.0,-1.0,-1.0,-1.0,-0.866667,0.296396,-0.751852,-0.666667,0.608761,0.793353,-1.0,-1.0,-1.0,-1.0,-0.866667,0.275187,-0.759259,-0.666667,0.707107,0.707107,-1.0,-1.0,-1.0,-1.0,-0.850000,0.284926,-0.762963,-0.666667,0.793353,0.608761,-1.0,-1.0,...,-1.0,-1.0,-0.700000,0.070663,-0.874074,-0.666667,0.000000,1.000000,-1.0,-1.0,-1.0,-1.0,-0.750000,0.070663,-0.874074,-0.666667,0.130526,0.991445,-1.0,-1.0,-1.0,-1.0,-0.783333,0.060058,-0.870370,-0.666667,0.258819,0.965926,-1.0,-1.0,-1.0,-1.0,-0.833333,0.050103,-0.866667,-0.666667,0.382683,0.923880,-1.0,-1.0
.

In [76]:
df_test = []

# 예시 코드
# 아직 유진이 어떻게 변수 추가할 지 몰라서 to_supervised는 쓰지 않았음.

for i in range(81):
    file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(1) + '.csv'
    new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(1) + '.csv'
    file_name = str(1) + '.csv'

    # adj_test에서 가져와야 하므로 new_path에서 불러온다
    temp = pd.read_csv(new_path)

    temp.drop('Unnamed: 0', axis = 1, inplace = True)
    testset = temp.copy()

    # 24로 저장한 24절기 season으로 이름 바꾸기
    testset.rename(columns = {'24' : 'season'}, inplace = True)

    # 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
    testset2 = drop_clms(testset)
    testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
    testset3 = cos_time(testset2)
    testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]

    testset5 = testset4.copy()
    testset5['GHI'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)
    testset6 = testset5[['DHI', 'DNI', 'WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']]
    testset7 = pd.DataFrame(scaler.transform(testset6))
    testset7.columns = testset6.columns

    testset8 = test_to_supervised(testset7, n_days)
    testset9 = testset8.iloc[-48:,:]
    df_test.append(testset9)

X_test = pd.concat(df_test)
X_test = pd.DataFrame(pca.transform(X_test), index = X_test.index)
# X_test = X_test.iloc[:, :n_obs]
X_test 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240
288,0.858608,13.196143,11.865798,1.707342,-0.319018,3.211001,-0.548020,-0.289071,-2.220812,0.394064,3.515840,-1.727938,1.875687,0.258097,0.732227,3.430281,-2.839604,2.014976,-1.088854,-1.356500,0.055601,-0.426138,0.101220,0.113705,0.021648,0.141876,0.619973,0.156060,1.134239,1.258100,-0.205531,-1.383118,-1.516281,-1.296285,-0.502518,-0.920388,0.828015,0.502474,0.974520,0.427612,...,-0.058586,0.128944,0.007955,-0.159257,-0.028375,0.053886,0.018613,-0.007660,0.069966,0.164109,-0.002492,-0.067431,-0.059604,-0.121160,-0.141430,-0.453497,-0.025890,0.024143,-0.014692,-0.034107,0.012269,-0.134811,-0.138601,0.030607,0.047036,0.066820,-0.139909,0.088527,0.002670,0.063700,0.123252,0.161596,-0.374635,0.489392,-0.017475,0.132829,0.005808,-0.109395,0.041354,-0.055919
289,-0.871715,13.198512,11.873385,1.718517,0.538922,3.178783,-0.555456,-0.331961,-2.222012,0.087771,3.413277,-1.949398,1.997560,-0.049006,0.383083,3.644881,-2.660269,1.810223,-1.227753,-1.220185,-0.124754,-0.431462,-0.071616,0.011488,-0.114244,-0.055090,0.681999,0.285230,0.998532,1.417405,0.642762,-1.527777,-1.237414,-1.370012,-0.314562,-0.950170,0.816127,0.399638,0.815206,0.398873,...,0.037497,-0.154742,-0.033461,-0.257175,0.059717,-0.174977,0.025230,0.114284,-0.006607,-0.095334,-0.003888,-0.007726,-0.130494,-0.007142,-0.297632,-0.443211,-0.082025,-0.003219,-0.031002,0.070505,0.029376,0.132512,-0.113509,-0.039987,-0.063330,-0.072361,-0.045322,-0.026377,-0.002987,-0.039167,-0.067760,0.198463,-0.056818,0.498005,0.044258,0.039168,0.020736,0.126538,-0.029838,0.061226
290,-2.587481,12.974506,11.880672,1.726139,1.360353,2.927262,-0.568754,-0.369857,-2.179435,-0.221678,3.293638,-2.135760,2.142211,-0.355187,0.062490,3.802077,-2.421329,1.548955,-1.397957,-1.066966,-0.281595,-0.440948,-0.243948,-0.082555,-0.236265,-0.233797,0.736633,0.389591,0.776997,1.205404,1.423372,-1.606825,-0.840241,-1.414902,-0.119591,-0.969140,0.813966,0.295503,0.571217,0.388306,...,-0.004328,0.051577,-0.185272,-0.329864,-0.005869,0.152500,-0.077622,-0.111828,-0.072154,-0.025177,-0.123171,0.077325,-0.084500,-0.002138,-0.240915,-0.519088,-0.025254,0.005483,0.051193,-0.029509,-0.100859,-0.096057,-0.047664,0.045171,0.012805,0.111189,0.071816,-0.019345,0.082031,0.050055,0.095315,0.216487,-0.239162,0.187797,0.007573,0.088140,0.002029,-0.107767,-0.038793,-0.108276
291,-4.258741,12.525588,11.887292,1.725736,2.091065,2.475942,-0.594202,-0.405220,-2.093237,-0.530682,3.163909,-2.277681,2.308859,-0.654990,-0.220032,3.903517,-2.131181,1.233651,-1.587853,-0.910476,-0.399893,-0.455441,-0.416451,-0.162059,-0.349510,-0.387265,0.778262,0.470653,0.561074,0.669781,1.973244,-1.606779,-0.361552,-1.434701,0.081187,-0.975570,0.797875,0.212340,0.271157,0.399896,...,-0.044008,0.007239,-0.010530,-0.388436,0.036135,-0.128619,-0.012382,0.036337,0.137677,0.071266,0.071632,0.129670,-0.046664,-0.089371,-0.189401,-0.528224,-0.091350,-0.012810,-0.072308,0.038987,0.119218,0.044006,0.020845,-0.073749,0.045986,-0.116700,0.235483,0.024025,0.034384,0.014477,-0.060082,-0.058283,-0.044859,0.227597,0.082950,0.009903,0.012557,0.101074,0.071377,0.061450
292,-5.857168,11.862237,11.893729,1.721649,2.682494,1.853093,-0.640767,-0.436490,-1.964870,-0.829117,3.022119,-2.366679,2.490831,-0.940888,-0.463593,3.947239,-1.791822,0.873915,-1.781200,-0.767774,-0.464556,-0.470157,-0.590110,-0.222044,-0.457243,-0.505569,0.802524,0.545285,0.420522,-0.053529,2.174341,-1.519292,0.158498,-1.430039,0.282274,-0.968429,0.756608,0.171703,-0.053908,0.426063,...,0.044200,-0.126855,-0.130667,-0.299777,0.016156,-0.021956,-0.005008,0.065525,-0.097109,-0.118424,-0.076547,0.072726,0.062047,0.056001,-0.349755,-0.409585,-0.029921,0.079402,0.030446,-0.010818,-0.115671,0.023036,0.056763,0.085508,-0.052813,0.057128,0.3

In [77]:
X_train_1, X_valid_1, Y_train_1,  Y_valid_1 = train_test_split(X, y_1, test_size=0.30, random_state=42)
X_train_2, X_valid_2, Y_train_2,  Y_valid_2 = train_test_split(X, y_2, test_size=0.30, random_state=42)

quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [81]:
# LGBM 모델을 구축하고 예측까지 하는 함수를 만든다. 
def LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test):
    
    # (a) 모델링
    model = LGBMRegressor(objective='quantile', alpha=q,
                         n_estimators=20000, bagging_fraction=0.7, learning_rate=0.005, save_binary = True, subsample=0.7, device = 'gpu')          
    
        # bagging_fraction : 배깅을 하기위해서 데이터를 랜덤 샘플링하여 학습에 사용한다. 비율은 0 < fraction <= 1 이며 0이 되지 않게 해야한다.
        # learning_rate : 일반적으로 0.01 ~ 0.1 정도로 맞추고 다른 파라미터를 튜닝한다. 나중에 성능을 더 높일 때 learning rate를 더 줄인다.
        # n_estimators : 기본값은 100, 1000 정도로 해주는 게 좋고 많을 수록 과적합이 발생한 가능성이 높음.
        # save_binary = True 넣어주면 더 빨라진다고 함.
        # https://greatjoy.tistory.com/72
        
    model.fit(X_train, Y_train, eval_metric = ['quantile'], eval_set=[(X_valid, Y_valid)], early_stopping_rounds=500, verbose=500)


        # verbose : eval metric이 이 숫자만큼의 round가 지난 다음 자동으로 출력된다.
        # early_stopping_rounds : 이 숫자가 가기 전까지 validation score가 증가하지 않으면 round를 멈춘다. 

    # (b) 예측
    pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

In [82]:
# Target 예측

def train_data(X_train, Y_train, X_valid, Y_valid, X_test):

    LGBM_models=[]
    LGBM_actual_pred = pd.DataFrame()

    for q in quantiles:
        print(q)
        pred, model = LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test)
        LGBM_models.append(model)
        LGBM_actual_pred = pd.concat([LGBM_actual_pred,pred],axis=1)

    LGBM_actual_pred.columns=quantiles
    
    return LGBM_models, LGBM_actual_pred

In [83]:
tick = time.time()

# Target1

models_1, results_1 = train_data(X_train_1, Y_train_1, X_valid_1, Y_valid_1, X_test)
results_1.sort_index()[:48]

0.1
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 1.46452
[1000]	valid_0's quantile: 1.4381
[1500]	valid_0's quantile: 1.42795
[2000]	valid_0's quantile: 1.42054
[2500]	valid_0's quantile: 1.41534
[3000]	valid_0's quantile: 1.41078
[3500]	valid_0's quantile: 1.40822
[4000]	valid_0's quantile: 1.40552
[4500]	valid_0's quantile: 1.40362
[5000]	valid_0's quantile: 1.40242
[5500]	valid_0's quantile: 1.40132
[6000]	valid_0's quantile: 1.39902
[6500]	valid_0's quantile: 1.39688
[7000]	valid_0's quantile: 1.39375
[7500]	valid_0's quantile: 1.39084
[8000]	valid_0's quantile: 1.38819
[8500]	valid_0's quantile: 1.386
[9000]	valid_0's quantile: 1.38435
[9500]	valid_0's quantile: 1.38283
[10000]	valid_0's quantile: 1.38129
[10500]	valid_0's quantile: 1.37985
[11000]	valid_0's quantile: 1.37884
[11500]	valid_0's quantile: 1.37775
[12000]	valid_0's quantile: 1.3771
[12500]	valid_0's quantile: 1.37605
[13000]	valid_0's quantile: 1.37521
[13500]	valid_0's quan

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.00


In [ ]:
# Target2

models_2, results_2 = train_data(X_train_2, Y_train_2, X_valid_2, Y_valid_2, X_test)
tock = time.time()
results_2.sort_index()

0.1
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 1.47593
[1000]	valid_0's quantile: 1.46746
[1500]	valid_0's quantile: 1.4672
[2000]	valid_0's quantile: 1.46719
Early stopping, best iteration is:
[1561]	valid_0's quantile: 1.46719
0.2
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 2.41129
[1000]	valid_0's quantile: 2.31542
[1500]	valid_0's quantile: 2.28496
[2000]	valid_0's quantile: 2.25748
[2500]	valid_0's quantile: 2.23815
[3000]	valid_0's quantile: 2.22304
[3500]	valid_0's quantile: 2.21114
[4000]	valid_0's quantile: 2.19901
[4500]	valid_0's quantile: 2.18751
[5000]	valid_0's quantile: 2.17657
[5500]	valid_0's quantile: 2.16957
[6000]	valid_0's quantile: 2.16217
[6500]	

In [ ]:
print('소요시간 :', tock-tick)

print(results_1.shape, results_2.shape)

In [ ]:
# 출력

submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results_1.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results_2.sort_index().values
submission2 = submission.set_index('id')
submission3 = not_minus(submission2)

In [ ]:
submission3.to_csv('submission11.csv')
!cp submission11.csv "drive/My Drive/"